In [0]:
from google.colab import drive
drive.mount('/content/drive')

# **TWO MODES OF WEB SCRAPING**

*   **BEAUTIFUL SOUP**
*   **NEWSPAPER3K** 



# Newspaper3k is a python library for web scraping. https://newspaper.readthedocs.io/en/latest/

In [0]:
pip install newspaper3k

# **IMPORTING REQUIRED LIBRARIES**

In [0]:
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from newspaper import Article
import numpy as np
import multiprocessing
import newspaper
import warnings
warnings.filterwarnings("ignore")
import requests
import nltk
import csv
import re

In [0]:
nltk.download('stopwords')
nltk.download('punkt')

# **LIST TO STRING CONVERTION FUNCTION**

In [0]:
def listToStr(s):
    string = ' '.join(map(str, s ))
    return string

# **NEWS WEBSITES USED :**

1.   https://www.nytimes.com/section/education
2.   https://www.reuters.com/finance
3.   https://www.foxnews.com/world
4.   https://www.news18.com
5.   https://www.bbc.com/news/technology
6. https://qz.com/africa/latest/
7. https://www.studentnewsdaily.com/archive/daily-news-article/







# **THE LIST IN WHICH ALL THE ARTICLES WILL BE APPENDED ONE AFTER THE OTHER FROM THE ABOVE MENTIONED WEBSITES**

In [0]:
articles = []

# **BEAUTIFUL SOUP LIBRARY FOR WEB SCRAPING - EXTRACTION OF ARTICLE LINKS WHICH WILL BE FURTHER USED TO FETCH THE SUMMARY OF EACH ARTICLE**

## **WEB SCRAPING OF ARTICLE LINKS FROM NEWYORK TIMES**

In [0]:
page = requests.get('https://www.nytimes.com/section/education') .text  #The GET method indicates that you’re trying to get or retrieve data from a specified resource
soup = BeautifulSoup(page, 'lxml')                                                              #Parsing the page content by removing the tags and other syntactical codes
href_links = []
for a in soup.find_all('a', href=True)[59:70]:                                             #Find all the hyperlinks inside the html tag <a>...</a>
    href_links.append(a['href'])

href_links
#alid_links = href_links[59:70]

In [0]:
for  site in href_links:                                           # The href links were extracted previously in the early part of the code you may check it.
    website_url = requests.get(site).text
    soup = BeautifulSoup(website_url, 'lxml')
    tags = soup.find_all('p')
    for tag in tags:
        articles.append(" ".join(tag.text.split()))

# **WEB SCRAPING OF ARTICLE LINKS FROM REUTERS NEWS WEBSITE**

In [0]:
try:
    html  = requests.get('https://www.reuters.com/finance') .text         #The GET method indicates that you’re trying to get or retrieve data from a specified resource
    soup = BeautifulSoup(html, 'lxml')                                                      #Parsing the page content by removing the tags and other syntactical codes
    links_list = []
    for li in soup.find_all('li', class_="nav-item")[0:8]:                              #Find all <li>..<\li> with class name "nav-item"
        links_list.append('https://www.reuters.com'+li.find("a", class_ = 'nav-link-sec')['href'])  #Extract the link inside the <li> with <a> tag.
except TypeError:
    pass

links_list

In [0]:
for  site4 in links_list:                                                       #The href links from reuters website is extracted in the early part of the code 
    website_url4 = requests.get(site4).text
    soup4 = BeautifulSoup(website_url4, 'lxml')
    tags4 = soup.find_all(['h3', 'p'])
    for tag4 in tags4:
        articles.append(" ".join(tag4.text.split()))

# **WEB SCRAPING OF A WEBSITE - QUARTZ AFRICA**

In [0]:
site2 = 'https://qz.com/africa/latest/'                 #This website did not require any href link extraction the content was directly available.
website_url2 = requests.get(site2).text
soup2 = BeautifulSoup(website_url2, 'lxml')
tags2 = soup2.find_all('h3')
for tag2 in tags2:
    articles.append(" ".join(tag2.text.split()))

# **WEB SCRAPING OF A WEBSITE - STUDENTS DAILY NEWS**

In [0]:
site3 = 'https://www.studentnewsdaily.com/archive/daily-news-article/'              #This website did not require any href link extraction the content was directly available.
website_url3 = requests.get(site3).text
soup3 = BeautifulSoup(website_url3, 'lxml')
tags3 = soup3.find_all('a')
for tag3 in tags3[30:529]:
    articles.append(" ".join(tag3.text.split()))

# **WEB SCRAPING OF ARTICLE LINKS FROM FOXNEWS WEBSITE**

In [0]:
webpage = requests.get('https://www.foxnews.com/world') .text
soup = BeautifulSoup(webpage, 'lxml')
links_list2 = []
class_names = ["menu-news",  "menu-opinion", "menu-news", "menu-politics", "menu-entertainment", "menu-business", "menu-lifestyle"]
for each_class in class_names:                                  # I could not extract required articles from the page so I have manually
                                                                                 # chosen the articles and represented as a lists in class_name.
    for li in soup.find_all('li', class_= each_class):
        links_list2.append('https://www.foxnews.com'+li.find("a")['href'])

# Converting list of list to flat lists
new_links_list2 = []
for i in links_list2:
       if i not in new_links_list2:
          new_links_list2.append(i)

new_links_list2


# **WEB SCRAPING OF ARTICLE LINKS FROM FOXNEWS USING NEWSPAPER3K**

The code snippets for writing to csv file, reading csv file, flattening of list, summary extraction and cleaning of summary will be repeating for each website. The reason that I did not create a function for it is, for analysing the output of each stage and checking if certain code snippents has to be applied on it. 

**THE CODING SAMPLES WERE TAKEN FROM -https://newspaper.readthedocs.io/en/latest/**

# Building a news source

In [0]:
#Building a news source
cnn_papers = newspaper.build('https://www.foxnews.com/world')  #

**Every news source has a set of recent articles. Hence every time you run this cell the currently updated articles will only be recovered. Hence there is a chance that we may end up fetching no news articles to avoid this I am uploading the article links to a csv file. Using the csv file we can extract the summary of each article.**

In [0]:
# Every news source has a set of recent articles. Hence every time you run this cell the currently updated articles will only be recovered.
# Hence there is a chance that we may end up fetching no news articles to avoid this I am uploading the article links to a csv file.
#Using the csv file we can extract the summary of each article.
articles_count = []
for article in cnn_papers.articles:
    articles_count.append(article.url)

In [0]:
articles_count

The return value of articles_count.size() changes from 400 to 2 because when we first crawled and we found 400
articles. However, on our second crawl, we eliminate all articles which have already been crawled.

# **WRITING THE LINKS TO THE CSV FILE <SAMPLE.CSV>**

In [0]:
with open('/content/drive/My Drive/sample.csv', 'w') as csvFile:
    writer = csv.writer(csvFile, quoting=csv.QUOTE_ALL)
    writer.writerow(articles_count)

csvFile.close()

# **READING THE CSV FILE TO FETCH THE ARTICLE LINKS**

In [0]:
with open('/content/drive/My Drive/FoxNews.csv', 'r') as f:
  reader = csv.reader(f)
  your_list = list(reader)

# **CONVERTING THE LIST OF LIST TO FLAT LIST**

In [0]:
flat_list = []
for sublist in your_list :
    for item in sublist:
        flat_list.append(item)

# **EXTRACTING THE SUMMARY OF EACH ARTICLE**

In [0]:
story = []
for every_article in flat_list:
    #print(every_article)
    article_name =  Article(url=every_article)      #Initializing an Article by itself
    article_name.download()                                  #Downloading an Article
    article_name.parse()                                       # Parsing an Article
    article_name.nlp()                                           # Extract out natural language properties from the text
    story.append(article_name.summary)              # Extracting the summary of the article

# **CLEANING THE SUMMARY BY REMOVING \n AND EMPTY STRINGS**

In [0]:
clean_story = []
for every_line in story:
    clean_story.append(every_line.replace("\n", ""))

clean_story = list(filter(None, clean_story))
clean_story

# **READING, LIST FLATTENING, SUMMARY EXTRACTION, CLEANING THE SUMMARY FOR THE FOX NEWS WEBSITE ON ANOTHER DAY**

In [0]:
with open('/content/drive/My Drive/FoxNews2.csv', 'r') as file:
  read = csv.reader(file)
  read_list = list(read)

In [0]:
flat_list_foxnews = []
for sub in read_list :
    for items in sub:
        flat_list_foxnews.append(items)

In [0]:
story_foxnews = []
for every_news in flat_list_foxnews:
    #print(every_article)
    article_nam =  Article(url=every_news)
    article_nam.download()
    article_nam.parse()
    article_nam.nlp()
    story_foxnews.append(article_nam.summary)

In [0]:
clean_story_foxnews = []
for every_line in story_foxnews:
    clean_story_foxnews.append(every_line.replace("\n", ""))

clean_story_foxnews = list(filter(None, clean_story_foxnews))
clean_story_foxnews

# **READING, LIST FLATTENING, SUMMARY EXTRACTION, CLEANING THE SUMMARY FOR THE NEWS18 WEBSITE**

THE SOURCE BUILDING AND ARTICLE EXTRACTION I HAVE PERFORMED IN ANOTHER NOTEBOOK. I DIDN'T ADD IT HERE BECAUSE IT WILL CREATE REDUNDANCY

In [0]:
with open('/content/drive/My Drive/News18.csv', 'r') as f:
  reader = csv.reader(f)
  your_list_News18 = list(reader)

In [0]:
flat_list_News18 = []
for sublist in your_list_News18 :
    for item in sublist:
        flat_list_News18.append(item)

In [0]:
story_News18 = []
for every_article in flat_list_News18:
    #print(every_article)
    article_name =  Article(url=every_article)
    article_name.download()
    article_name.parse()
    article_name.nlp()
    story_News18.append(article_name.summary)

In [0]:
clean_story_news18 = []
for every_line in story_News18:
    clean_story_news18.append(every_line.replace("\n", ""))

clean_story_news18 = list(filter(None, clean_story_news18))
clean_story_news18

# **SOURCE BUILDING, ARTICLE EXTRACTION, CREATING AND READING CSV FILES, LIST FLATTENING, SUMMARY EXTRACTION, CLEANING THE SUMMARY FOR THE BBC NEWS WEBSITE**

In [0]:
bbc_news = newspaper.build('https://www.bbc.com/news/technology', memorize_articles = False)

In [0]:
articles_bbc = []
for article in bbc_news.articles:
    articles_bbc.append(article.url)

In [0]:
articles_bbc

In [0]:
with open('/content/drive/My Drive/sample.csv', 'w') as CsvFile:
    writercsv = csv.writer(CsvFile, quoting=csv.QUOTE_ALL)
    writercsv.writerow(articles_bbc)

CsvFile.close()

In [0]:
with open('/content/drive/My Drive/bbc_news.csv', 'r') as fs:
  read_bbc = csv.reader(fs)
  your_list_bbc = list(read_bbc)

In [0]:
flat_list_bbc = []
for sublist in your_list_bbc :
    for item in sublist:
        flat_list_bbc.append(item)

In [0]:
story_bbc = []
for every_article in flat_list_bbc:
    #print(every_article)
    article_name =  Article(url=every_article)
    article_name.download()
    article_name.parse()
    article_name.nlp()
    story_bbc.append(article_name.summary)

In [0]:
story_bbc

In [0]:
clean_story_bbc = []
for every_line in story_bbc:
    clean_story_bbc.append(every_line.replace("\n", ""))

clean_story_bbc = list(filter(None, clean_story_bbc))
clean_story_bbc

# **COMBINING ALL THE ARTICLES FROM VARIOUS WEBSITES USED INTO ONE LIST FOR FURTHER PROCESSING**

In [0]:
# COMBINING ALL THE ARTICLES FROM VARIOUS WEBSITES USED INTO ONE LIST FOR FURTHER PROCESSING
articles = articles + clean_story + clean_story_news18 + clean_story_foxnews + clean_story_bbc

articles
print(len(articles))

# **ELIMINATING IDENTICAL NEWS CONTENT**

In [0]:
new_articles = []
for i in articles:
       if i not in new_articles:
          new_articles.append(i)

len(new_articles)

# **SPLITTING OF THE NEWS CONTENT INTO TOKENS - TOKENIZATION**

In [0]:
tokenizer = RegexpTokenizer(r'\w+')                     #Regex expression for allowing group of alphanumeric characters and underscore
tokenized_articles = tokenizer.tokenize(listToStr(new_articles))
print(tokenized_articles)
print(len(tokenized_articles))

# **ELIMINATION OF STOPWODS FOR FASTER EXECUTION**

In [0]:
cachedStopWords = stopwords.words("english")
stopwords_tokenized = [word for word in tokenized_articles if word not in cachedStopWords]
print(stopwords_tokenized)
print(len(stopwords_tokenized))

# **USING GENSIM PYTHON LIBRARY FOR SIMILARITY RETRIEVAL FROM THE DATASET**

**HYPER PARAMETERS**


1.   min_count - Ignores all words with total frequency lower than this.
2.   size - Dimensionality of the word vectors.
3. window - Maximum distance between the current and predicted word within a sentence.
4. sg - Training algorithm: 1 for skip-gram; otherwise CBOW.



In [0]:
import gensim
SKIP_GRAM = gensim.models.Word2Vec([stopwords_tokenized], min_count = 1, size = 50, 
                                             window = 2, sg = 1)

In [0]:
bias_words = ['abuse', 'victim', 'dissidents', 'abortion', 'black', 'violence', 'segregated', 'poorer', 'dehumanized', 'indigenous']

#**IDEALOGY**
Bias Word Extraction. Given the list of seed words, we extract
a larger number of bias words using the Wikipedia dataset of latest articles7, from which we compute word embeddings using
word2Vec with the skip-gram model. In the next step we exploit
the semantic relationships of word vectors to automatically extract
bias words given the seed words and a measure of distance between
word vectors. Mikolov et al.[12] showed that within the word2Vec
vector space similar words are grouped close to each other because
they often appear in similar context. A trivial approach would be to
simply extract the closest words for every seed word. In this case,
if the seed word is a bias word, we would presumably retrieve bias
words but also words that are related to the given seed word but
are not bias words. For example for the seed word “charismatic”
we find the word “preacher” among the closest words in the vector
space.
To improve the extraction, we make use of another property of
word2Vec. Instead of extracting the closest words of only one word,
we compute the mean of multiple seed words in the vector space
and extract the closest words for the resulting vector. This helps us
to identify clusters of bias words.

REFERENCE - Detecting Biased Statements in Wikipedia
Christoph Hube and Besnik Fetahu
L3S Research Center, Leibniz University of Hannover
Hannover, Germany
{hube, fetahu}@L3S.de

**FLOWCHART:-**
BIAS WORD (BLACK) --->  FIND CLUSTER OF 10 SIMILAR WORDS ---> FIND THE VECTOR REPRESENTATION OF EACH OF THE WORDS IN THE CLUSTER ---> CALCULATE THE MEAN OF WORD VECTORS --->  EXTRACT 100 SIMILAR WORDS TO THE  MEAN OF THE WORD VECTORS ---> COMPUTE THE COSINE DISTANCE BETWEEN THE BIAS WORD AND THE SIMILAR WORDS OBTAINED. 
# **DETERMINING MOST SIMILAR WORDS TO A GIVEN WORD**

In [0]:
def most_similar(bias_word):
    similar_word = SKIP_GRAM.wv.most_similar(bias_word)
    return similar_word

# **THE OUPUT WILL CONTAIN A TUPLE OF WORD THE SIMILARITY INDEX BUT THE FUNCTION EXTRACTS THE WORD FROM THE TUPLE LEAVING THE INDEX VALUE**

In [0]:
def extract_word_alone(word_to_process):
    list_of_words = most_similar(word_to_process)
    word_alone = [lis[0] for lis in list_of_words] 
    return word_alone

# **EXTRACT THE VECTOR REPRESENTATION OF CLUSTER OF 10 SIMILAR WORDS OF A SINGLE BIAS WORD**

In [0]:
def capture_word_vectors(word_to_find):
    word_vector = []    
    similar_word_alone = extract_word_alone(word_to_find)
    for i in range(len(similar_word_alone)):
        word_vector.append(SKIP_GRAM[similar_word_alone[i]]) 
    return word_vector

# **BIAS WORD INPUT AND CALCULATION OF THE AVERAGE OF WORD VECTORS OF THE CLUSTER OF SIMILAR WORDS TO THE INPUT WORD - HERE "BLACK"**

In [0]:
def avg_vec(test_word):
    vector = capture_word_vectors(test_word)
    mean_of_vec = np.mean( np.array([vector[0], vector[1], vector[2], vector[3],vector[4], vector[5], vector[6], vector[7], vector[8], vector[9]]), axis=0 )
    return mean_of_vec

# **USING THE MEAN OF WORD VECTORS COMPUTED FROM THE CLUSTER OF SIMILAR WORDS WE CAN DETERMINE THE MOST SIMILAR WORDS BY VECTOR.**
**TOP N = 100, TO EXTRACT 100 SIMILAR WORDS OF A CLUSTER OF WORDS FOR A BIAS WORD INPUT OF MY CHOICE**

In [0]:
def find_words_SimByVec(test_word):
    similar_to_BiasWord = SKIP_GRAM.wv.similar_by_vector(avg_vec(test_word), topn=100, restrict_vocab=None)
    top100_word_alone = [lis[0] for lis in similar_to_BiasWord] 
    print("The top 100 words similar to the bias words are", top100_word_alone)
    print("\n")
    return top100_word_alone

# **COMPUTE THE COSINE DISTANCE BETWEEN THE BIAS WORD AND THE SIMILAR WORD COMPUTED FROM CLUSTER OF WORDS**

In [0]:
def cosine_similarity(test_word):
    top_words = find_words_SimByVec(test_word)
    for i in top_words: 
    #SKIP_GRAM.wv.similarity('black', i )
        print("The cosine distance between the word {} and it's similar word {} is = {}".format(test_word, i, SKIP_GRAM.wv.similarity(test_word, i ) ) )

# **FINAL FUNCTION TO PRINT THE TOP 100 WORDS AND THE COSINE SIMILARITY BETWEEN THE BIAS WORD AND ITS SIMILAR WORD**

## **BIAS WORDS FROM THE DATASET = ['abuse', 'victim', 'dissidents', 'abortion', 'black', 'violence', 'segregated', 'poorer', 'dehumanized', 'indigenous']**

# COPY PASTE EACH WORD AT A TIME AND PASTE IT IN THE BELOW FUNCTION "cosine_similarity(word)". IT WILL GENERATE THE TOP 100 SIMILAR WORDS AND ITS COSINE SIMILARITY

In [0]:
cosine_similarity('dehumanized')

# **ROUGH WORK DO NOT EVALUATE**

In [0]:
def printing(test_word):
    print(test_word)

In [0]:
printing("singh")

In [0]:
seed_word_alone

In [0]:
seed_word_1 = SKIP_GRAM.wv.most_similar("African")

In [0]:
seed_word_alone = [lis[0] for lis in seed_word_1] 

In [0]:
list_of_words = most_similar("abuse")

In [0]:
words_of_similar = extract_word_alone(list_of_words)

In [0]:
words_of_similar[0]

In [0]:
similar_word_abuse =        SKIP_GRAM.wv.most_similar("abuse")
similar_word_victim =        SKIP_GRAM.wv.most_similar("victim")
similar_word_dissidents = SKIP_GRAM.wv.most_similar("dissidents")
similar_word_abortion =    SKIP_GRAM.wv.most_similar("abortion")
similar_word_black =         SKIP_GRAM.wv.most_similar("black")
#similar_word_abuse = SKIP_GRAM.wv.most_similar("abuse")
#similar_word_abuse = SKIP_GRAM.wv.most_similar("abuse")
#similar_word_abuse = SKIP_GRAM.wv.most_similar("abuse")
#similar_word_abuse = SKIP_GRAM.wv.most_similar("abuse")
#similar_word_abuse = SKIP_GRAM.wv.most_similar("abuse")

In [0]:
vec_word1 = SKIP_GRAM['memory']
vec_word2 = SKIP_GRAM['fault']

In [0]:
vec_word1 

In [0]:
vec_word2

In [0]:
import numpy as np

In [0]:
mean_vec = np.mean( np.array([vec_word1, vec_word2]), axis=0 )

In [0]:
mean_vec

In [0]:
SKIP_GRAM.wv.similar_by_vector(mean_vec, topn=100, restrict_vocab=None)

In [0]:
num_features = 300
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel. more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 5


# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [0]:
model_SKGram = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
)

In [0]:
model_SKGram.build_vocab(tokenized_articles)

In [0]:
model_SKGram.train(tokenized_articles, total_examples=len(tokenized_articles), epochs=5)

In [0]:
model_SKGram.most_similar("countries")

In [0]:
try_articles1 = []
for  site in href_links:
    website_url = requests.get(site).text
    soup = BeautifulSoup(website_url, 'lxml')
    for match in soup.findAll('span'):
        match.unwrap()
        


In [0]:
tags = soup.find_all('p')
for tag in tags:
    try_articles1.append(" ".join(tag.text.split()))

In [0]:
try_articles1

In [0]:
soup = BeautifulSoup(website_url, 'lxml')
    tags = soup.find_all('p')
    for tag in tags:
        try_articles.append(" ".join(tag.text.split()))

In [0]:
tokenized_articless = [[]]
tokenizer = RegexpTokenizer(r'\w+')
for item in range(len(new_articles)):
    tokenized_articless.append(tokenizer.tokenize(new_articles[item]))
tokenized_articless

In [0]:
model_ted = Word2Vec(sentences=tokenized_articless, size=100, window=5, min_count=1, workers=4, sg=1)
model_ted.wv.most_similar("provider")

In [0]:
articles2=[]
site = 'https://qz.com/africa/latest/'
website_url = requests.get(site).text
soup = BeautifulSoup(website_url, 'lxml')
tags = soup.find_all('h3')
for tag in tags:
    articles2.append(" ".join(tag.text.split()))

In [0]:
articles2

In [0]:
page = requests.get('https://qz.com/africa/latest') 
soup = BeautifulSoup(page.content, 'html.parser')

In [0]:
weblinks = soup.find_all('article')

In [0]:
weblinks

In [0]:
pagelinks = []
for link in weblinks[5:]:    
      url = link.contents[0].find_all('a')[0]   
      pagelinks.append('http://qz.com'+url.get('href'))

In [0]:
pagelinks

In [0]:
page = requests.get('https://www.studentnewsdaily.com/archive/daily-news-article/') 
soup = BeautifulSoup(page.content, 'html.parser')

In [0]:
articles3 = []
site = 'https://www.studentnewsdaily.com/archive/daily-news-article/'
website_url = requests.get(site).text
soup = BeautifulSoup(website_url, 'lxml')
tags = soup.find_all('a')
for tag in tags[30:529]:
    articles3.append(" ".join(tag.text.split()))

In [0]:
list(enumerate(articles3))